# Feature Engineering
One approach s to determine the level of variation across Chicago in terms of temperature for a single day. If the variation is not huge, and if it appears to not be correlated with crime, then we can simply take global temperature indicators for each day and assign them to the crime records. 

Out of 356 days per year, we have 13 stations with roughly 250 weather points for the year. Let's see if that means we don't have weather points for the weekends and holidays. This would be a major blocker for this project, because in reality, most of the crime in Chicago happens on extremely hot weekends. After some analysis, we realized this was not a limitation in our project.

Due to the complexity in the join, at this preliminary stage we're OK with just keeping the average temperature. If we run into accuracy problems with our model later, we can circle back and re-think how we're computing the temperate. Let's just settle with the average over those 13 stations, and use that for each crime. This means that rather than using the GeoPandas Data Frames and joining the 2 sets, we can just append temperature data to the crimes data. This is good news, because the pandas joined needed a new package installation that gave us HUGE issues.

We should NOT write over the old date. It contains information about the time of day that the crime was committed, which we will want to keep for other modeling purposes. We'll just add a new column for reformatted dates so we can join it with the weather averages. This function will take a few minutes to run.

# ETL as a single process
Eventually, it's very helpful to have a single place for your ETL strategy code. This lets you easily change it as new data comes in or new ideas that might enhance your model are discovered. Then, after your end result is built up, you can work backwards from it and change your modeling strategy based on its actual behavior in the real world.

In [34]:
import pandas as pd
from datetime import date
from dateutil.rrule import rrule, DAILY

def reformat_crime_date(date_string):

    items = date_string.split("/")
    year = items[-1]
    month = items[0]
    day = items[1]    
    
    new_date = "{}-{}-{}".format(year, month, day)
        
    return new_date

def get_mapper(crime_days):

    d = {}

    for each in crime_days:
        old_date = each.split()[0]
        if old_date not in d:
            new_date = reformat_crime_date(old_date[:])
            d[old_date] = new_date

    return d

def get_average_by_day(weather_df):

    days_in_2018 = get_days_from_range([2018, 1, 1], [2018, 9, 30])
    
    rt = {}
    
    for d in days_in_2018:

        # now, for each day, grab every TMAX
        rows = weather_df.loc[ weather_df["DATE"] == d]
        rt[d] = rows["TMAX"].mean() 
    return rt


def get_mapping_dict():

    rt = {}

    a = date(2018, 1, 1)
    b = date(2018, 9, 30)

    day_of_week = {0: "Monday", 1:"Tuesday", 2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}

    for dt in rrule(DAILY, dtstart=a, until=b):
        day_of_year = dt.strftime("%Y-%m-%d")
        n = dt.weekday()
        day = day_of_week[n]
        rt[day_of_year] = day
        
    return rt

def add_day_of_week(df):

    for idx, row in df.iterrows():
        day_of_year = row["DATE"]
        day_of_week = m[day_of_year]
        df.loc[idx, "Day of Week"] = day_of_week
        
    return df


def get_days_from_range(day_1, day_2):

    rt = []
    
    a = date(day_1[0], day_1[1], day_1[2])
    b = date(day_2[0], day_2[1], day_2[2])

    for dt in rrule(DAILY, dtstart=a, until=b):
        day_of_year = dt.strftime("%Y-%m-%d")
        rt.append(day_of_year)
    return rt


In [22]:
def get_updated_data(weather_file, crime_file):
    
    # read into memory
    weather_df = pd.read_csv(weather_file)
    crime_df = pd.read_csv(crime_file, index_col = "ID")
    
    # add new columns
    weather_df["Day Of Week"] = [" " for i in range(weather_df.shape[0])]    
    crime_df["Reformatted Date"] = [" " for i in range(crime_df.shape[0])]

    # get day of week for weather
    weather_df = add_day_of_week(weather_df)
    
    # compute averages by day
    
    return weather_df, crime_df


In [ ]:
def modify_crime_dates(df):
    mapper = get_mapper(crime_df["Date"].unique().tolist()[:])

    for idx, row in crime_df.iterrows():

        new_date = mapper[row["Date"].split()[0]]

        crime_df.loc[idx, "Reformatted Date"] = new_date

In [29]:
def main(weather_file, crime_file):
    '''
    The primary function for execution. This does the following:
        (1) Reads weather and crime data into memory
        (2) Adds a Day of the week to the weather data set
        (3) Builds a mapper to reformat the crime dates
        (4) 

    '''
    weather_df, crime_df = get_updated_data(weather_file, crime_file)

#     crime_df = modify_crime_dates(crime_df)
    
    crime_df = pd.read_csv("../Data/crimes_reformatted.csv")

    
main("../Data/cleaned_weather_2018.csv", "../Data/crimes_2018_reduced.csv")

In [27]:
# crime_df.to_csv("../Data/crimes_reformatted.csv")

In [41]:
# def add_avg_temp_to_crime(weather_df, crime_df):

averages = get_average_by_day(weather_df)

crime_df["AVG TEMP"] = [0.0 for i in range(crime_df.shape[0])]

for idx, row in crime_df.iterrows():
    day = row["Reformatted Date"]
    temp = averages[day]
    crime_df.loc[idx, "AVG TEMP"] = temp

    
    
    
# add_avg_temp_to_crime(weather_df, crime_df)

In [42]:
crime_df.to_csv("../Data/crime_with_temp.csv")